In [ ]:
!pip install datasets evaluate transformers[sentencepiece] seqeval sacrebleu huggingface_hub accelerate

In [ ]:
from datasets import load_dataset
import evaluate
from huggingface_hub import notebook_login
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import warnings

warnings.filterwarnings('ignore')
notebook_login()

In [ ]:
lang1, lang2 = "en", "ja"
raw_datasets = load_dataset("kde4", lang1=lang1, lang2=lang2)
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=8192)
split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
print(split_datasets)

In [ ]:
for i in range(20):
  print(split_datasets["train"][i])

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-tatoeba-en-ja"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [ ]:
def preprocess_function(examples):
    inputs = [ex[lang1] for ex in examples["translation"]]
    targets = [ex[lang2] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

In [ ]:
print(tokenized_datasets)

In [ ]:
for i in range(20):
  print(tokenized_datasets["train"][i])

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-ja",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(max_length=128)

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")